# Constant market (Benchmark-adjusted CRRA)

In [49]:
import pandas as pd
import numpy as np
from scipy.integrate import quad
import itertools

# Fixed parameters
t = 0
W = 1000
delta = 0.02

r = 0.04
sigma = 0.2
_lambda = 0.3
epsilon = 0  # tolerance level for checking closeness to benchmark

# Functions
def F(t, T, r, alpha, beta):
    return (1 - np.exp(-(r + beta - alpha)*(T - t))) / (r + beta - alpha)

def g(t, T, r, delta, gamma):
    exponent = (delta / gamma) + (1 - 1/gamma) * r + (1/(2 * gamma)) * (1 - 1/gamma) * (_lambda ** 2)
    return (1 - np.exp(-exponent * (T - t))) / exponent

def compute_benchmark_adjusted_crra(h, gamma_H, T, alpha, beta, eps):
    f_val = F(t, T, r, alpha, beta)

    # Initial assumption: use gamma_H and test region
    g_H = g(t, T, r, delta, gamma_H)
    C_t_candidate = h + (1 / g_H) * (W - h * f_val)

    # Determine region based on epsilon-band around h
    if abs(C_t_candidate - h) < eps:
        region = 'high'
        gamma_used = gamma_H * 2
        g_used = g(t, T, r, delta, gamma_used)
        C_t_star = h - (1 / g_used) * (W - h * f_val)
    else:
        region = 'low'
        gamma_used = gamma_H
        g_used = g_H
        C_t_star = C_t_candidate

    aktieandel = (_lambda / (gamma_used * sigma)) * (W - h * f_val) / W
    cash_position = 1 - aktieandel

    return aktieandel, cash_position, C_t_star, region, gamma_used

# Parameter ranges
hs = [20]
gammas = [1,2,5,10]
Ts = [50]
alphas = [0]
betas = [0]

# Simulation
results = []

for h, gamma, T, alpha, beta in itertools.product(hs, gammas, Ts, alphas, betas):
    aktieandel, cash_pos, C_star, region, gamma_used = compute_benchmark_adjusted_crra(
        h, gamma, T, alpha, beta, epsilon
    )

    results.append({
        'h (C_bar)': h,
        'gamma base': gamma,
        'T': T,
        'epsilon': epsilon,
        'Used gamma': gamma_used,
        'Region': region,
        'Stock %': round(aktieandel * 100, 2),
        'Cash %': round(cash_pos * 100, 2),
        'C_t*': round(C_star, 2),
    })

df_results = pd.DataFrame(results)
df_results

,h (C_bar),gamma base,T,epsilon,Used gamma,Region,Stock %,Cash %,C_t*
0,20,1,50,0,1,low,85.15,14.85,37.96
1,20,2,50,0,2,low,42.58,57.42,46.83
2,20,5,50,0,5,low,17.03,82.97,47.72
3,20,10,50,0,10,low,8.52,91.48,47.19


# Stochastic market (Benchmark-adjusted CRRA)

In [1]:
import pandas as pd
import numpy as np
from scipy.integrate import quad
import itertools

# Fixed parameters
t = 0
W = 1000
delta = 0.02
kappa = 0.3
r = 0.04

sigma = 0.2
Sigma = 0.2  # sigma_lambda
Lambda = 0.3
Lambda_cap = 0.3

epsilon = 0 # tolerance for checking closeness to benchmark

# Auxiliary functions
def F(t, T, r, alpha, beta):
    return (1 - np.exp(-(r + beta - alpha)*(T - t))) / (r + beta - alpha)

def safe_divide(numerator, denominator):
    if np.isclose(denominator, 0):
        return 0
    else:
        return numerator / denominator

def stochastic_model_A0_A1_A2(theta, b, t, T, kappa, Lambda_cap, Sigma, gamma):
    def A0(t, T):
        term1 = -(delta / gamma) * (T - t)
        term2 = -r * (1 - 1/gamma) * (T - t)
        term3 = -0.5 * np.log((2 * theta - (theta - b) * (1 - np.exp(-2 * theta * (T - t)))) / (2 * theta))
        term4 = ((1 - gamma) / (2 * gamma**2)) * (
            ((kappa**2 * Lambda_cap**2) / theta**2 + Sigma**2 / (theta + b)) * (T - t) +
            ((kappa**2 * Lambda_cap**2) / theta**3) *
            ((theta - 2 * b) * np.exp(-2 * theta * (T - t)) + 4 * b * np.exp(-theta * (T - t)) - theta - 2 * b) /
            (2 * theta - (theta - b) * (1 - np.exp(-2 * theta * (T - t))))
        )
        return term1 + term2 + term3 + term4

    def A1(t, T):
        return ((1 - gamma) / gamma**2) * (kappa * Lambda_cap / theta) * ((1 - np.exp(-theta * (T - t)))**2) / \
               (2 * theta - (theta - b) * (1 - np.exp(-2 * theta * (T - t))))

    def A2(t, T):
        return ((1 - gamma) / gamma**2) * (1 - np.exp(-2 * theta * (T - t))) / \
               (2 * theta - (theta - b) * (1 - np.exp(-2 * theta * (T - t))))
    
    return A0, A1, A2

def numerator_D1(A0, A1, A2, t, T, Lambda, alpha, beta, gamma):
    integrand = lambda s: (A1(t, s) + A2(t, s) * Lambda) * (1 + alpha * F(s, T, r, alpha, beta))**(1 - 1/gamma) * \
               np.exp(A0(t, s) + A1(t, s) * Lambda + 0.5 * A2(t, s) * Lambda**2)
    result, _ = quad(integrand, t, T, limit=200)
    return result

def denominator_D1(A0, A1, A2, t, T, Lambda, alpha, beta, gamma):
    integrand = lambda s: (1 + alpha * F(s, T, r, alpha, beta))**(1 - 1/gamma) * \
               np.exp(A0(t, s) + A1(t, s) * Lambda + 0.5 * A2(t, s) * Lambda**2)
    result, _ = quad(integrand, t, T, limit=200)
    return result

def compute_benchmark_adjusted_stochastic(h, gamma_H, T, alpha, beta, eps):
    f_val = F(t, T, r, alpha, beta)

    # Initial guess using gamma_H
    b = kappa - (1 - 1/gamma_H) * Sigma
    theta = np.sqrt(b**2 - (Sigma**2 * (1 - gamma_H)) / gamma_H**2)
    A0, A1, A2 = stochastic_model_A0_A1_A2(theta, b, t, T, kappa, Lambda_cap, Sigma, gamma_H)

    g_lambda = denominator_D1(A0, A1, A2, t, T, Lambda, alpha, beta, gamma_H)
    C_t_candidate = h + (W - h * f_val) / g_lambda  # Correct C_t_candidate formula

    if abs(C_t_candidate - h) < eps:
        region = 'high'
        gamma_used = gamma_H * 2
    else:
        region = 'low'
        gamma_used = gamma_H

    # Recompute with gamma_used if needed
    b = kappa - (1 - 1/gamma_used) * Sigma
    theta = np.sqrt(b**2 - (Sigma**2 * (1 - gamma_used)) / gamma_used**2)
    A0, A1, A2 = stochastic_model_A0_A1_A2(theta, b, t, T, kappa, Lambda_cap, Sigma, gamma_used)

    D1_val = numerator_D1(A0, A1, A2, t, T, Lambda, alpha, beta, gamma_used) / denominator_D1(A0, A1, A2, t, T, Lambda, alpha, beta, gamma_used)

    myopic = Lambda / (gamma_used * sigma) * (W - h * f_val) / W
    hedging = -(Sigma / sigma) * D1_val * (W - h * f_val) / W

    total_weight = myopic + hedging
    cash_position = 1 - total_weight
    myopic_hedging_ratio = safe_divide(hedging, myopic)

    # Final C_t* (after adjustment)
    g_lambda_final = denominator_D1(A0, A1, A2, t, T, Lambda, alpha, beta, gamma_used)
    C_t_star = h + (W - h * f_val) / g_lambda_final

    return myopic, hedging, total_weight, cash_position, myopic_hedging_ratio, region, gamma_used, C_t_candidate, C_t_star

# Example simulation
hs = [5]
gammas = [5]
Ts = [10]
alphas = [0]
betas = [0]

results = []

for h, gamma, T, alpha, beta in itertools.product(hs, gammas, Ts, alphas, betas):
    myopic, hedging, total_weight, cash_position, ratio, region, gamma_used, C_t_candidate, C_t_star = compute_benchmark_adjusted_stochastic(
        h, gamma, T, alpha, beta, epsilon
    )
    results.append({
        'h (C_bar)': h,
        'gamma base': gamma,
        'T': T,
        'epsilon': epsilon,
        'Used gamma': gamma_used,
        'Region': region,
        'Myopic weight (%)': round(myopic * 100, 2),
        'Hedging weight (%)': round(hedging * 100, 2),
        'Total weight (%)': round(total_weight * 100, 2),
        'Cash weight (%)': round(cash_position * 100, 2),
        'Hedging/Myopic ratio': round(ratio, 2),
        'C_t candidate': round(C_t_candidate, 2),
        'C_t* (final)': round(C_t_star, 2),
    })

df_results = pd.DataFrame(results)
df_results


,h (C_bar),gamma base,T,epsilon,Used gamma,Region,Myopic weight (%),Hedging weight (%),Total weight (%),Cash weight (%),Hedging/Myopic ratio,C_t candidate,C_t* (final)
0,5,5,10,0,5,low,28.76,17.53,46.29,53.71,0.61,129.58,129.58
